In [11]:
# Import elastic search (Install with `pip3 installl elasticsearch`)
from elasticsearch import Elasticsearch
# Define the running elasticsearch server (by default localhost:9200)
client = Elasticsearch('localhost')

In [ ]:
# This notebook covers simple use of elasticsearch and data mitigation from mysql to elasticsearch

In [13]:
# Define mappings or 'schema' for index 'mdas'
mappings = {
    'mappings':{
            'properties':{
                #the number
                'number':{'type':'keyword' },
                #name of user
                'name':{'type':'keyword'},
                #trust points
                'tPoint' : {'type':'float'}
            }
        }
}
client.indices.create(index='mdas',body=mappings)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'mdas'}

In [18]:
#inserting data to mdas
docs = [
    {
        'number': '+60129304840',
        'name': 'Reza',
        'tPoint': 7.09
    }
]

for doc in docs:
    client.create(index='mdas',id=0,body=doc)
    

In [24]:
# Get all data from index 'mdas'
query = {
    'query':{
        'match_all':{}
    }
}
client.search(index='mdas',body=query)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'mdas',
    '_type': '_doc',
    '_id': '0',
    '_score': 1.0,
    '_source': {'number': '+60129304840', 'name': 'Reza', 'tPoint': 7.09}}]}}

In [29]:
# Get data related to +60129304840
query = {
    'query':{
        'bool':{
            'must':{
                'term':{
                    'number':'+60129304840'
                }
            }
        }
    }
}
client.search(index='mdas',body=query)

{'took': 8,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1, 'relation': 'eq'},
  'max_score': 0.2876821,
  'hits': [{'_index': 'mdas',
    '_type': '_doc',
    '_id': '0',
    '_score': 0.2876821,
    '_source': {'number': '+60129304840', 'name': 'Reza', 'tPoint': 7.09}}]}}

In [75]:
# Delete data for Reza
query = {
    'query' :{
        'bool':{
            'must':{
                'term':{
                    'name':'Reza'
                }
            }
        }
    }
}
client.delete_by_query(index='mdas',body=query)

{'took': 20,
 'timed_out': False,
 'total': 6,
 'deleted': 6,
 'batches': 1,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [31]:
# Get data related to +60129304840
query = {
    'query':{
        'bool':{
            'must':{
                'term':{
                    'number':'+60129304840'
                }
            }
        }
    }
}
client.search(index='mdas',body=query)

{'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [66]:
# import data from mysql to elasticsearch
# retrieve data from mysql 
import pymysql
import pymysql.cursors
names=[]
numbers=[]
con = pymysql.connect('localhost', 'root', 'password', 'telco1', charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)
cur = con.cursor()
cur.execute("select * from individual")
rows = cur.fetchall()
for row in rows:
    names.append(row["name"])
    numbers.append(row["number"])


In [67]:
names

['Ricardo', 'Richard', 'Anna', 'Kath', 'Leroy', 'Fluffy']

In [68]:
numbers

['+6020494091',
 '+60129393830',
 '+601293903021',
 '+6037948498',
 '+6037948498',
 '+60129304840']

In [71]:
# generate random float for trust point
import random
def Tgen():
    x = random.uniform(0,10)
    return x


In [84]:
# insert data to elasticsearch 
i = 0 
for name in names:
    tpoint=Tgen()
    print("Adding {}, {}, {}".format(numbers[i],name,tpoint))
    docs = {'number': '{}'.format(numbers[i]),'name': '{}'.format(name),'tPoint': '{}'.format(tpoint)}
    client.create(index='mdas',id=i,body=docs)
    i=i+1

Adding +6020494091, Ricardo, 4.760578585241354
Adding +60129393830, Richard, 7.097847218888003
Adding +601293903021, Anna, 9.047318488415726
Adding +6037948498, Kath, 1.4790370147982501
Adding +6037948498, Leroy, 2.4407688070644107
Adding +60129304840, Fluffy, 3.475786851380125


In [85]:
# Get all data from index 'mdas'
query = {
    'query':{
        'match_all':{}
    }
}
client.search(index='mdas',body=query)

{'took': 362,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'mdas',
    '_type': '_doc',
    '_id': '0',
    '_score': 1.0,
    '_source': {'number': '+6020494091',
     'name': 'Ricardo',
     'tPoint': '4.760578585241354'}},
   {'_index': 'mdas',
    '_type': '_doc',
    '_id': '1',
    '_score': 1.0,
    '_source': {'number': '+60129393830',
     'name': 'Richard',
     'tPoint': '7.097847218888003'}},
   {'_index': 'mdas',
    '_type': '_doc',
    '_id': '2',
    '_score': 1.0,
    '_source': {'number': '+601293903021',
     'name': 'Anna',
     'tPoint': '9.047318488415726'}},
   {'_index': 'mdas',
    '_type': '_doc',
    '_id': '3',
    '_score': 1.0,
    '_source': {'number': '+6037948498',
     'name': 'Kath',
     'tPoint': '1.4790370147982501'}},
   {'_index': 'mdas',
    '_type': '_doc',
    '_id': '4',
    '_score': 1.0,
    '_source': {